In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [5]:
train_dir = 'data/splited/train/'
test_dir = 'data/splited/test/'
val_dir = 'data/splited/val/'

In [14]:
def load_data(directory):
    images = []
    labels = []
    label_to_int = {}  # Dictionary to map labels to integers
    int_label = 0  # Starting integer label
    for label in os.listdir(directory):
        label_to_int[label] = int_label
        label_path = os.path.join(directory, label)
        for image_file in os.listdir(label_path):
            image_path = os.path.join(label_path, image_file)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (224, 224))  # Resize if necessary
            images.append(image)
            labels.append(int_label)
        int_label += 1
    return np.array(images), np.array(labels)

In [15]:
X_train, y_train = load_data(train_dir)
X_test, y_test = load_data(test_dir)
X_val, y_val = load_data(val_dir)

In [16]:
X_train = X_train / 255.0
X_test = X_test / 255.0
X_val = X_val / 255.0

MemoryError: Unable to allocate 68.1 GiB for an array with shape (60745, 224, 224, 3) and data type float64

In [17]:
num_classes = 10

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
y_val = to_categorical(y_val, num_classes)

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [19]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [20]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
1899/1899 [==============================] - 1741s 915ms/step - loss: 1.9277 - accuracy: 0.6205 - val_loss: 0.8941 - val_accuracy: 0.7238
Epoch 2/10
1899/1899 [==============================] - 1778s 936ms/step - loss: 0.7539 - accuracy: 0.7605 - val_loss: 0.8600 - val_accuracy: 0.7445
Epoch 3/10
1899/1899 [==============================] - 1767s 931ms/step - loss: 0.5450 - accuracy: 0.8244 - val_loss: 0.8581 - val_accuracy: 0.7352
Epoch 4/10
1899/1899 [==============================] - 1764s 929ms/step - loss: 0.4009 - accuracy: 0.8694 - val_loss: 0.9249 - val_accuracy: 0.7318
Epoch 5/10
1899/1899 [==============================] - 1752s 922ms/step - loss: 0.3117 - accuracy: 0.8989 - val_loss: 1.0327 - val_accuracy: 0.7415
Epoch 6/10
1899/1899 [==============================] - 1747s 920ms/step - loss: 0.2462 - accuracy: 0.9215 - val_loss: 1.3463 - val_accuracy: 0.7337
Epoch 7/10
1899/1899 [==============================] - 1743s 918ms/step - loss: 0.2113 - accuracy: 0.9343

In [21]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


226/226 [==============================] - 45s 200ms/step - loss: 1.8489 - accuracy: 0.7127
Test Loss: 1.8488743305206299
Test Accuracy: 0.7126596570014954


In [22]:
model.save('great_model_with_spectogram.h5')

C:\Programs\python\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
if isinstance(model.layers[-1], Activation) and model.layers[-1].activation == softmax:
    model = tf.keras.Model(model.input, model.layers[-2].output, name=model.name)
else:
    print('Error: last layer is not SoftMax Activation')

Error: last layer is not SoftMax Activation


In [33]:
%pip install qualia_codegen_core
import qualia_codegen_core
from qualia_codegen_core.graph.KerasModelGraph import KerasModelGraph
from qualia_codegen_core.graph.Quantization import Quantization
from qualia_codegen_core.graph.RoundMode import RoundMode

from importlib.resources import files
main_path = str((files('qualia_codegen_core.examples')/'Linux'/'main.cpp').resolve())

In [34]:
modelgraph = KerasModelGraph(model).convert()
print(modelgraph)

—————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
Inputs                                           | Layer                                            | Outputs                                          | Input shape                                      | Output shape                                    
—————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
                                                 | conv2d_input                                     | conv2d                                           | (1, 224, 224, 3)                                 | ((1, 224, 224, 3),)                

In [35]:
import copy
from pathlib import Path

float_modelgraph = copy.deepcopy(modelgraph)

# layer quantization annotations for float32
for node in float_modelgraph.nodes:
    # No scale factor if not fixed-point quantization on integers
    node.q = Quantization(
            number_type=float,
            width=32,
            long_width=32,
            weights_scale_factor=0,
            output_scale_factor=0,
            weights_round_mode=RoundMode.NONE,
            output_round_mode=RoundMode.NONE,
            )

float_res = qualia_codegen_core.Converter(output_path=Path('gsc_output_floating')).convert_model(float_modelgraph)

with open('gsc_model_floating.h', 'w') as f:
    f.write(float_res)

Graphviz not available
Softmax activation must be used as a standalone layer, not combined to another layer (TDenseLayer)
ModelGraph validation failed


TypeError: write() argument must be str, not None

In [29]:
!g++ -std=c++17 -Wall -Wextra -pedantic -Ofast -o gsc_floating -include gsc_output_floating/include/defines.h -Igsc_output_floating/include gsc_output_floating/model.c {main_path}
!./gsc_floating x_test_gsc_250.csv y_test_gsc_250.csv

cc1plus.exe: fatal error: gsc_output_floating/model.c: No such file or directory
compilation terminated.
<command-line>: fatal error: gsc_output_floating/include/defines.h: No such file or directory
compilation terminated.
'.' is not recognized as an internal or external command,
operable program or batch file.


In [30]:
fixed_modelgraph = copy.deepcopy(modelgraph)

# layer quantization annotations for int16 Q9.7
for node in fixed_modelgraph.nodes:
    node.q = Quantization(
            number_type=int,
            width=16,
            long_width=32,
            weights_scale_factor=7,
            output_scale_factor=7,
            weights_round_mode=RoundMode.FLOOR,
            output_round_mode=RoundMode.FLOOR,
            )

fixed_res = qualia_codegen_core.Converter(output_path=Path('gsc_output_fixed')).convert_model(fixed_modelgraph)

with open('gsc_model_fixed.h', 'w') as f:
    f.write(fixed_res)

Graphviz not available
Softmax activation must be used as a standalone layer, not combined to another layer (TDenseLayer)
ModelGraph validation failed


TypeError: write() argument must be str, not None

In [31]:
!g++ -std=c++17 -Wall -Wextra -pedantic -Ofast -o gsc_fixed -include gsc_output_fixed/include/defines.h -Igsc_output_fixed/include gsc_output_fixed/model.c {main_path}
!./gsc_fixed x_test_gsc_250.csv y_test_gsc_250.csv

cc1plus.exe: fatal error: gsc_output_fixed/model.c: No such file or directory
compilation terminated.
<command-line>: fatal error: gsc_output_fixed/include/defines.h: No such file or directory
compilation terminated.
'.' is not recognized as an internal or external command,
operable program or batch file.
